In [23]:
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

from lib.Utility import exportExcelWithTimeStamp

from datetime import date, timedelta
import yfinance as yf #Alternative package if webreader does not work: pip install yfinance
import numpy as np # Fundamental package for scientific computing with Python
import joblib
import plotly.express as px
from plotly import graph_objects as go
# Train the model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import os

In [24]:
# Funzione per calcolare la forma recente escludendo la partita in corso
def calculate_recent_form(df, window=5):
    df['HomePointsPrevious'] = df['HomePoints'].shift(1)
    df['AwayPointsPrevious'] = df['AwayPoints'].shift(1)
    
    df['HomeForm'] = df.groupby('Home')['HomePointsPrevious'].rolling(window).sum().reset_index(level=0, drop=True)
    df['AwayForm'] = df.groupby('Away')['AwayPointsPrevious'].rolling(window).sum().reset_index(level=0, drop=True)
    
    # Elimina le colonne temporanee
    df = df.drop(columns=['HomePointsPrevious', 'AwayPointsPrevious'])
    
    return df

# Funzione per calcolare la forma recente in casa e in trasferta escludendo la partita in corso
def calculate_recent_home_away_form(df, window=5):
    df['HomePointsPrevious'] = df['HomePoints'].shift(1)
    df['AwayPointsPrevious'] = df['AwayPoints'].shift(1)
    
    df['HomeRecentHomeForm'] = df.groupby('Home')['HomePointsPrevious'].rolling(window).sum().reset_index(level=0, drop=True)
    df['AwayRecentAwayForm'] = df.groupby('Away')['AwayPointsPrevious'].rolling(window).sum().reset_index(level=0, drop=True)
    
    # Elimina le colonne temporanee
    df = df.drop(columns=['HomePointsPrevious', 'AwayPointsPrevious'])
    
    return df

# Funzione per calcolare la probabilità attesa
def expected_probability(elo_a, elo_b):
    return 1 / (1 + 10**((elo_b - elo_a) / 400))

# Funzione per aggiornare il punteggio Elo
def update_elo(elo, result, expected, k=30):
    return elo + k * (result - expected)

# Funzione per calcolare il punteggio Elo per ogni partita
def calculate_elo(df, k=30, initial_elo=1500):
    # Inizializza i punteggi Elo per le squadre
    elo_dict = {}
    
    # Liste per memorizzare il punteggio Elo prima della partita
    elo_home_list = []
    elo_away_list = []
    
    # Itera attraverso ogni riga del DataFrame
    for index, row in df.iterrows():
        home_team = row['Home']
        away_team = row['Away']

        
        # Se la squadra non ha un Elo, assegnagli l'elo iniziale
        if home_team not in elo_dict:
            elo_dict[home_team] = initial_elo
        if away_team not in elo_dict:
            elo_dict[away_team] = initial_elo
            
        # Ottieni i punteggi Elo prima della partita
        elo_home = elo_dict[home_team]
        elo_away = elo_dict[away_team]
        
        # Calcola la probabilità attesa di vittoria
        expected_home = expected_probability(elo_home, elo_away)
        expected_away = 1 - expected_home
        
        # Memorizza gli Elo attuali
        elo_home_list.append(elo_home)
        elo_away_list.append(elo_away)
        
        # Calcola il risultato della partita (1 per vittoria, 0.5 per pareggio, 0 per sconfitta)
        if row['HG'] > row['AG']:
            result_home = 1
            result_away = 0
        elif row['HG'] < row['AG']:
            result_home = 0
            result_away = 1
        else:
            result_home = 0.5
            result_away = 0.5
        
        # Aggiorna i punteggi Elo
        new_elo_home = update_elo(elo_home, result_home, expected_home, k)
        new_elo_away = update_elo(elo_away, result_away, expected_away, k)
        
        # Aggiorna il dizionario Elo
        elo_dict[home_team] = new_elo_home
        elo_dict[away_team] = new_elo_away
    
    # Aggiungi le colonne con gli Elo al DataFrame
    df['elo_home'] = elo_home_list
    df['elo_away'] = elo_away_list
    
    return df

In [25]:
# Funzione per calcolare i punti in base al risultato
def calculate_points(result):
    if result == 'H':
        return 3, 0
    elif result == 'A':
        return 0, 3
    else:
        return 1, 1

# Funzione per calcolare i risultati delle ultime 3 partite
def calculate_last_3_matches(df, team, is_home=True):
    points = []
    goals_scored = []
    goals_conceded = []
    
    if is_home:
        mask = (df['Home'] == team)
        points = df.loc[mask, 'HomePoints']
        goals_scored = df.loc[mask, 'HG']
        goals_conceded = df.loc[mask, 'AG']
    else:
        mask = (df['Away'] == team)
        points = df.loc[mask, 'AwayPoints']
        goals_scored = df.loc[mask, 'AG']
        goals_conceded = df.loc[mask, 'HG']
        
    # Calcolo dei punti delle ultime 3 partite
    last_3_points = points.rolling(3).sum().shift(1)
    
    # Calcolo della media dei goal fatti nelle ultime 3 partite
    avg_goals_scored = goals_scored.rolling(3).mean().shift(1)
    
    # Calcolo della media dei goal subiti nelle ultime 3 partite
    avg_goals_conceded = goals_conceded.rolling(3).mean().shift(1)
    
    # Calcolo della media esponenziale pesata dei punti nelle ultime 3 partite
    ewma_points = points.ewm(span=3).mean().shift(1)
    
    # Calcolo della media esponenziale pesata dei goal fatti nelle ultime 3 partite
    ewma_goals_scored = goals_scored.ewm(span=3).mean().shift(1)
    
    # Calcolo della media esponenziale pesata dei goal subiti nelle ultime 3 partite
    ewma_goals_conceded = goals_conceded.ewm(span=3).mean().shift(1)
    
    return last_3_points, avg_goals_scored, avg_goals_conceded, ewma_points, ewma_goals_scored, ewma_goals_conceded



In [26]:
# Funzione per aggiungere le colonne richieste
def add_gap_columns(df):

    # Controlla che le colonne richieste esistano
    required_columns = ['HomePointsCumulative', 'AwayPointsCumulative', 'HomeGoalsCumulative', 'AwayGoalsCumulative']
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"La colonna richiesta '{col}' non è presente nel file Excel.")
    
    # Calcola le nuove colonne
    df['HomePointGap'] = df['HomePointsCumulative'] - df['AwayPointsCumulative']
    df['AwayPointGap'] = df['AwayPointsCumulative'] - df['HomePointsCumulative']
    df['HomeGoalGap'] = df['HomeGoalsCumulative'] - df['AwayGoalsCumulative']
    df['AwayGoalGap'] = df['AwayGoalsCumulative'] - df['HomeGoalsCumulative']
    return df    

In [27]:
 # Funzione per ottenere l'ultimo scontro diretto
def get_last_match_result(row, data):
    # Trova tutti gli scontri diretti precedenti a quello corrente
    past_matches = data[((data['Home'] == row['Home']) & (data['Away'] == row['Away'])) |
                        ((data['Home'] == row['Away']) & (data['Away'] == row['Home']))]
    
    # Filtra solo le partite avvenute prima della partita corrente
    past_matches = past_matches[past_matches['Date'] < row['Date']]
    
    if not past_matches.empty:
        last_match = past_matches.iloc[-1]
        
        # Determina il risultato rispetto alla partita corrente
        if last_match['Home'] == row['Home']:
            return last_match['Res']  # Il risultato dell'ultimo scontro è corretto rispetto alla partita attuale
        else:
            # Se le squadre sono invertite, inverti anche il risultato
            if last_match['Res'] == 'H':
                return 'A'
            elif last_match['Res'] == 'A':
                return 'H'
            else:
                return 'D'
    else:
        return None  # Se non ci sono scontri diretti precedenti, ritorna None

In [28]:
cssFolders = []#['data/Bundesliga']
for folder_path in cssFolders:
    for filename in os.listdir(folder_path):
        if(filename.endswith('.csv')):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path)
            xlsx_file = file_path.replace('.csv', '.xlsx')
            df.to_excel(xlsx_file, index=False, engine='openpyxl')

### Feature Engineering

In [29]:
# Specifica il percorso della cartella
folder_paths = ['data/Esotic']

# Itera su tutti i file nella cartella
for folder_path in folder_paths:
    for filename in os.listdir(folder_path):
        if(filename.endswith('.xlsx')):
            file_path = os.path.join(folder_path, filename)
                # Verifica se il percorso è un file (e non una cartella)
            if os.path.isfile(file_path):
                print(f"Processing file: {file_path}")

            data = pd.read_excel(file_path)
            # Crea una lista vuota per memorizzare i dataframe di ogni stagione
            all_seasons_data = []
            # Ottieni l'elenco delle stagioni uniche
            seasons = data["Season"].unique()

            # Iterazione per ogni stagione
            for season in seasons:
                print(f"Processing season: {season}")
                season_data = data.copy()
                season_data = season_data[season_data["Season"] == season]
                # Supponiamo che il DataFrame si chiami data
                # Aggiungere le colonne per i goal cumulativi
                season_data['HomeGoalsCumulative'] = 0
                season_data['AwayGoalsCumulative'] = 0

                # Creare un dizionario per tenere traccia dei goal cumulativi di ogni squadra
                goals_cumulative = {}

                # Iterare sulle righe del season_dataFrame
                for index, row in season_data.iterrows():
                    home_team = row['Home']
                    away_team = row['Away']
                    home_goals = row['HG']
                    away_goals = row['AG']
                    
                    # Inizializzare il conteggio dei goal per le squadre se non già presente
                    if home_team not in goals_cumulative:
                        goals_cumulative[home_team] = 0
                    if away_team not in goals_cumulative:
                        goals_cumulative[away_team] = 0
                    
                    # Assegnare i goal cumulativi fino a quel momento
                    season_data.at[index, 'HomeGoalsCumulative'] = goals_cumulative[home_team]
                    season_data.at[index, 'AwayGoalsCumulative'] = goals_cumulative[away_team]
                    
                    # Aggiornare i goal cumulativi con i goal della partita attuale
                    goals_cumulative[home_team] += home_goals
                    goals_cumulative[away_team] += away_goals

                import pandas as pd

                # Supponiamo che il season_dataFrame si chiami season_data
                # Aggiungere le colonne per i punti cumulativi
                season_data['HomePointsCumulative'] = 0
                season_data['AwayPointsCumulative'] = 0

                # Creare due dizionari per tenere traccia dei punti cumulativi di ogni squadra
                points_cumulative = {}

                # Iterare sulle righe del season_dataFrame
                for index, row in season_data.iterrows():
                    home_team = row['Home']
                    away_team = row['Away']
                    result = row['Res']
                    
                    # Inizializzare i punti per le squadre se non già presenti
                    if home_team not in points_cumulative:
                        points_cumulative[home_team] = 0
                    if away_team not in points_cumulative:
                        points_cumulative[away_team] = 0
                    
                    # Assegnare i punti cumulativi fino a quel momento
                    season_data.at[index, 'HomePointsCumulative'] = points_cumulative[home_team]
                    season_data.at[index, 'AwayPointsCumulative'] = points_cumulative[away_team]
                    
                    # Aggiornare i punti cumulativi in base al risultato della partita
                    if result == 'H':  # Vittoria della squadra di casa
                        points_cumulative[home_team] += 3
                    elif result == 'A':  # Vittoria della squadra ospite
                        points_cumulative[away_team] += 3
                    elif result == 'D':  # Pareggio
                        points_cumulative[home_team] += 1
                        points_cumulative[away_team] += 1

                # Supponiamo che il season_dataFrame si chiami season_data
                # Aggiungere le colonne per i goal subiti cumulativi
                season_data['HomeGoalsConcededCumulative'] = 0
                season_data['AwayGoalsConcededCumulative'] = 0

                # Creare un dizionario per tenere traccia dei goal subiti cumulativi di ogni squadra
                goals_conceded_cumulative = {}

                # Iterare sulle righe del season_dataFrame
                for index, row in season_data.iterrows():
                    home_team = row['Home']
                    away_team = row['Away']
                    home_goals = row['HG']  # Goal fatti dalla squadra di casa
                    away_goals = row['AG']  # Goal fatti dalla squadra ospite
                    
                    # Inizializzare i goal subiti per le squadre se non già presenti
                    if home_team not in goals_conceded_cumulative:
                        goals_conceded_cumulative[home_team] = 0
                    if away_team not in goals_conceded_cumulative:
                        goals_conceded_cumulative[away_team] = 0
                    
                    # Assegnare i goal subiti cumulativi fino a quel momento
                    season_data.at[index, 'HomeGoalsConcededCumulative'] = goals_conceded_cumulative[home_team]
                    season_data.at[index, 'AwayGoalsConcededCumulative'] = goals_conceded_cumulative[away_team]
                    
                    # Aggiornare i goal subiti cumulativi con i goal della partita attuale
                    goals_conceded_cumulative[home_team] += away_goals  # La squadra di casa subisce i goal della squadra ospite
                    goals_conceded_cumulative[away_team] += home_goals  # La squadra ospite subisce i goal della squadra di casa

                season_data['MatchGoal'] = season_data['HG'] + season_data['AG']

                # Calcolo della differenza di punti tra le due squadre
                season_data['PointsDifference'] = abs(season_data['HomePointsCumulative'] - season_data['AwayPointsCumulative'])

                # Calcolo del rapporto tra i goal fatti e subiti per la squadra di casa
                season_data['HomeGoalsRatio'] = season_data['HomeGoalsCumulative'] / season_data['HomeGoalsConcededCumulative']

                # Calcolo del rapporto tra i goal fatti e subiti per la squadra in trasferta
                season_data['AwayGoalsRatio'] = season_data['AwayGoalsCumulative'] / season_data['AwayGoalsConcededCumulative']

                # Calcolo della differenza tra i goal fatti della squadra di casa rispetto a quelli della squadra in trasferta
                # season_data['GoalsDifference'] = abs(season_data['HomeGoalsCumulative'] - season_data['AwayGoalsCumulative'])
                season_data['GoalsDifference'] = (season_data['HomeGoalsCumulative'] - season_data['AwayGoalsCumulative'])

                # Calcolo della differenza tra i goal subiti dalla squadra di casa rispetto a quelli subiti dalla squadra in trasferta
                season_data['ConcededGoalsDifference'] = abs(season_data['HomeGoalsConcededCumulative'] - season_data['AwayGoalsConcededCumulative'])

                season_data['HomePoints'] = season_data.apply(lambda x: 3 if x['HG'] > x['AG'] else (1 if x['HG'] == x['AG'] else 0), axis=1)
                season_data['AwayPoints'] = season_data.apply(lambda x: 3 if x['AG'] > x['HG'] else (1 if x['HG'] == x['AG'] else 0), axis=1)

                season_data = calculate_recent_form(season_data)
                season_data = calculate_recent_home_away_form(season_data)
                season_data = calculate_elo(season_data)
                season_data['EloRatio'] = season_data['elo_home']/season_data['elo_away']



                # Calcola i valori per ogni squadra
                for team in season_data['Home'].unique():
                    # Per le squadre in casa
                    home_values = calculate_last_3_matches(season_data, team, is_home=True)
                    season_data.loc[season_data['Home'] == team, 'HomeLast3Points'] = home_values[0]
                    season_data.loc[season_data['Home'] == team, 'HomeAvgGoalsScored'] = home_values[1]
                    season_data.loc[season_data['Home'] == team, 'HomeAvgGoalsConceded'] = home_values[2]
                    season_data.loc[season_data['Home'] == team, 'HomeEwmaPoints'] = home_values[3]
                    season_data.loc[season_data['Home'] == team, 'HomeEwmaGoalsScored'] = home_values[4]
                    season_data.loc[season_data['Home'] == team, 'HomeEwmaGoalsConceded'] = home_values[5]

                    # Per le squadre in trasferta
                    away_values = calculate_last_3_matches(season_data, team, is_home=False)
                    season_data.loc[season_data['Away'] == team, 'AwayLast3Points'] = away_values[0]
                    season_data.loc[season_data['Away'] == team, 'AwayAvgGoalsScored'] = away_values[1]
                    season_data.loc[season_data['Away'] == team, 'AwayAvgGoalsConceded'] = away_values[2]
                    season_data.loc[season_data['Away'] == team, 'AwayEwmaPoints'] = away_values[3]
                    season_data.loc[season_data['Away'] == team, 'AwayEwmaGoalsScored'] = away_values[4]
                    season_data.loc[season_data['Away'] == team, 'AwayEwmaGoalsConceded'] = away_values[5]

                # Calcola il numero di partite vinte, perse e pareggiate
                season_data['HomeWins'] = season_data['HomePoints'].rolling(3).apply(lambda x: (x == 3).sum()).shift(1)
                season_data['HomeDraws'] = season_data['HomePoints'].rolling(3).apply(lambda x: (x == 1).sum()).shift(1)
                season_data['HomeLosses'] = season_data['HomePoints'].rolling(3).apply(lambda x: (x == 0).sum()).shift(1)

                season_data['AwayWins'] = season_data['AwayPoints'].rolling(3).apply(lambda x: (x == 3).sum()).shift(1)
                season_data['AwayDraws'] = season_data['AwayPoints'].rolling(3).apply(lambda x: (x == 1).sum()).shift(1)
                season_data['AwayLosses'] = season_data['AwayPoints'].rolling(3).apply(lambda x: (x == 0).sum()).shift(1)

            

                # Crea la nuova colonna 'UltimoScontroDiretto'
                season_data['UltimoScontroDiretto'] = season_data.apply(lambda row: get_last_match_result(row, season_data), axis=1)


                season_data['Last3PointsDifference'] = season_data['HomeLast3Points'] - season_data['AwayLast3Points']
                season_data['GoalRatioDifference'] = season_data['HomeGoalsRatio'] - season_data['AwayGoalsRatio']
                season_data['EwmaGoalsSum'] = season_data['HomeEwmaGoalsScored'] + season_data['HomeEwmaGoalsConceded'] + season_data['AwayEwmaGoalsScored'] + season_data['AwayEwmaGoalsConceded']
                season_data['GoalsSum'] = season_data['HomeGoalsCumulative'] + season_data['AwayGoalsCumulative'] + season_data['HomeGoalsConcededCumulative'] + season_data['AwayGoalsConcededCumulative']
                season_data = add_gap_columns(season_data)
                all_seasons_data.append(season_data)

            resultFileName = 'data/engdata/'+ folder_path[5:] + '/' + filename[:-5] + '-E.xlsx'
            final_data = pd.concat(all_seasons_data, ignore_index=True)
            final_data.to_excel(resultFileName, index=False)

Processing file: data/Esotic\ARG.xlsx
Processing season: 2012/2013
Processing season: 2013/2014
Processing season: 2014
Processing season: 2015
Processing season: 2016
Processing season: 2016/2017
Processing season: 2017/2018
Processing season: 2018/2019
Processing season: 2019/2020
Processing season: 2020
Processing season: 2021
Processing season: 2022
Processing season: 2023
Processing season: 2024
Processing file: data/Esotic\BRA.xlsx
Processing season: 2012
Processing season: 2013
Processing season: 2014
Processing season: 2015
Processing season: 2016
Processing season: 2017
Processing season: 2018
Processing season: 2019
Processing season: 2020
Processing season: 2021
Processing season: 2022
Processing season: 2023
Processing season: 2024
Processing file: data/Esotic\CHN.xlsx
Processing season: 2014
Processing season: 2015
Processing season: 2016
Processing season: 2017
Processing season: 2018
Processing season: 2019
Processing season: 2020
Processing season: 2021
Processing seaso

## TODO
- Average Goal Per Match H/A
